In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import regex as re

In [5]:
filename = "covid_vaccine"

In [6]:
path = "../datasets/covid_vaccine/covid_vaccine.csv"
df = pd.read_csv(path).drop("Unnamed: 0", axis=1)[["video_id", "video_title", "video_transcript"]]
df.head()

,video_id,video_title,video_transcript
0,im3otpqYAiQ,Covid Vaccine Study Finds Links to Health Cond...,[CC may contain inaccuracies] In terms of how...
1,uiwjAj0zfKQ,If You Get All 5 COVID Vaccines,and all right we're done now if you're feelin...
2,LfmhYVCCGhc,Joe Rogan says tons of people &quot;died sudde...,Speaker 1: This is really painful to watch. O...
3,SU_SSfiYtfM,New COVID booster coming soon,what are those our new fall arsenal of vaccin...
4,7MAlEYqWUTk,Being Nice to Anti-Vaxxers,so you're against the covert vaccine oh here ...


In [7]:
patterns = []

# Removing occurances of \xa0 and \n
patterns.append(r"\xa0|\n")

# Removing text enclosed in brackets
patterns.append(r"\[\w+\]")

# Replacing stray '000's to 'thousands'
patterns.append(r"(?<=\s)000(?=\s)")

# Mistranscriptions of the word 'COVID'
patterns.append(r"(?<=\s)co(ve(r)?t|id)(?=\s)")

# Mistranscriptions of the word 'COVID-19'
patterns.append(r"(?<=\s)coveted(?=\s)")

# Removing forward slash from escaped apostrophes
patterns.append(r"\\(?=\')")

# Replacing '%' with the word 'percent'
patterns.append(r"(?<=\d)\%")

# Removing 'Speaker %d:' occurances
patterns.append(r"Speaker\s\d\:")

# Removing '[\xa0__\xa0]'
patterns.append(r"\[\\xa0\_\_\\xa0\]")

patterns

['\\xa0|\\n',
 '\\[\\w+\\]',
 '(?<=\\s)000(?=\\s)',
 '(?<=\\s)co(ve(r)?t|id)(?=\\s)',
 '(?<=\\s)coveted(?=\\s)',
 "\\\\(?=\\')",
 '(?<=\\d)\\%',
 'Speaker\\s\\d\\:',
 '\\[\\\\xa0\\_\\_\\\\xa0\\]']

In [9]:
from textblob import TextBlob

def textblob_subjectivity(transcript):
    for entry in transcript:
        testimonial = TextBlob(entry)
        print(testimonial.sentiment.subjectivity)
        

In [10]:
textblob_subjectivity(df["video_transcript"])

0       [CC may contain inaccuracies] In terms of how...
1       and all right we're done now if you're feelin...
2       Speaker 1: This is really painful to watch. O...
3       what are those our new fall arsenal of vaccin...
4       so you're against the covert vaccine oh here ...
                             ...                        
145     was the Pfizer covet vaccine tested on stoppi...
146     >> WE HAVE REACTION FROM THAT >> WE HAVE REAC...
147     from micro TV this is beyond the Noise episod...
148     A bill has been introduced in the Idaho legis...
149     The federal COVID-19 public health emergency ...
Name: video_transcript, Length: 150, dtype: object
0.4479338842975206
0       [CC may contain inaccuracies] In terms of how...
1       and all right we're done now if you're feelin...
2       Speaker 1: This is really painful to watch. O...
3       what are those our new fall arsenal of vaccin...
4       so you're against the covert vaccine oh here ...
                  

TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'float'>